In [ ]:
import os
import sys
import torch
import torch._dynamo
import torch.utils.cpp_extension
base_dir = os.environ.get('TORCHSIM_DIR', default='/workspace/PyTorchSim')
sys.path.append(base_dir)

from Scheduler.scheduler import PyTorchSimRunner
module = PyTorchSimRunner.setup_device()
device = module.custom_device()

def test_result(name, out, cpu_out, rtol=1e-4, atol=1e-4):
    if torch.allclose(out.cpu(), cpu_out, rtol=rtol, atol=atol):
        message = f"|{name} Test Passed|"
        print("-" * len(message))
        print(message)
        print("-" * len(message))
    else:
        message = f"|{name} Test Failed|"
        print("-" * len(message))
        print(message)
        print("-" * len(message))
        print("custom out: ", out.cpu())
        print("cpu out: ", cpu_out)
        exit(1)

def test_exponent2(device, size=(128, 128)):
    def exponent2(a):
        return a.exp2()
    x = torch.randn(size).to(device=device)
    opt_fn = torch.compile(dynamic=False)(exponent2)
    res = opt_fn(x)
    out = exponent2(x.cpu())
    test_result("exponent2", res, out)

Using /root/.cache/torch_extensions/py310_cu121 as PyTorch extensions root...
Emitting ninja build file /root/.cache/torch_extensions/py310_cu121/npu/build.ninja...
Building extension module npu...
Allowing ninja to set a default number of workers... (overridable by setting the environment variable MAX_JOBS=N)
Loading extension module npu...


ninja: no work to do.


In [2]:
input = torch.randn(16, 16)
npu_x = input.to(device=device)
cpu_x = input.to("cpu")
func = torch.exp2
opt_fn = torch.compile(dynamic=False)(func)
npu_out = opt_fn(npu_x)
cpu_out = func(cpu_x)
test_result("exp2", npu_out, cpu_out)

[2025-12-03 02:02:14,679] [0/0] torch._inductor.debug: [WARNING] model___9 debug trace: /tmp/torchinductor_root/uu/cuumxtbdv4ukzpymchmrda2exohouwcdybawmj2v7jog4vbvoycf.debug


Wrapper Codegen Path = /tmp/torchinductor_root/uu/cuumxtbdv4ukzpymchmrda2exohouwcdybawmj2v7jog4vbvoycf.py
[Gem5] Gem5 is running... 
[Spike] Running Spike simulator
[TOGSim] TOGSim is running..  
[TOGSim] Simulation log is stored to "/workspace/PyTorchSim/togsim_results/20251203_020218.log"
------------------
|exp2 Test Passed|
------------------


In [3]:
!cat /tmp/torchinductor_root/uu/cuumxtbdv4ukzpymchmrda2exohouwcdybawmj2v7jog4vbvoycf.py


from ctypes import c_void_p, c_long
import torch
import math
import random
import os
import tempfile
from math import inf, nan
from torch._inductor.hooks import run_intermediate_hooks
from torch._inductor.utils import maybe_profile
from torch._inductor.codegen.memory_planning import _align as align

from torch import device, empty, empty_strided
from PyTorchSimFrontend.extension_codecache import CustomAsyncCompile
from PyTorchSimFrontend.extension_config import CONFIG_SRAM_BUFFER_PLAN, CONFIG_TOGSIM_EAGER_MODE
from Simulator.simulator import TOGSimulator
from PyTorchSimFrontend.extension_op import sparse_mm_dummy_stonne_outer
from torch._inductor.select_algorithm import extern_kernels

aten = torch.ops.aten
inductor_ops = torch.ops.inductor
assert_size_stride = torch._C._dynamo.guards.assert_size_stride
alloc_from_pool = torch.ops.inductor._alloc_from_pool
reinterpret_tensor = torch.ops.aten._reinterpret_tensor
custom_async_compile = CustomAsyncCompile()
os.environ["TORCHSIM_LAST_COMP